# 오늘의 날씨 메일링 봇

매일 정해진 시간(7시 무렵) 오늘의 날씨 정보를 메일로 보냅니다.
이후 13시, 19시에 현재 기온을 유머러스한 멘트와 함께 보냅니다. 하하하.
맨 아래는 테스트용 코드입니다.

* 현 코드는 가산동만 가능, 함수 mission(e6)의 target 변수 변경 시 다른 지역 가능함.

In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1278 sha256=c4a7e5c647b438a045934c45b88eb517814816996a4c5cc976cdbb81da6867f7
  Stored in directory: /home/ubuntu/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [2]:
!pip install requests

In [5]:
!pip install urllib3

In [45]:
#크롤링을 위한 lib
from bs4 import BeautifulSoup
import requests
import re
#메일을 위한 lib
import smtplib
from email.mime.text import MIMEText
#자동화를 위한 lib
import time
import schedule
from datetime import datetime

# 미션 수행 함수

In [93]:
def todayWeather(dongName):
    head = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="
    keywords = dongName
    tail = "+날씨&oquery=오늘의+날씨&tqi=UCGJgwp0JXVss4ffedNssssstHo-344270"
    url = head + keywords + tail
    html = requests.get(url).text
    bs = BeautifulSoup(html, "lxml")
    minMax = bs.select_one(" .merge").get_text()
    sensible = bs.select_one(" .sensible").get_text()
    uv = bs.select_one(" .indicator").get_text()
    mise = bs.select(" .indicator")[1].select("dd")[0].get_text()
    choMise = bs.select(" .indicator")[1].select("dd")[1].get_text()
    ozone = bs.select(" .indicator")[1].select("dd")[2].get_text()
    castText = bs.select(" .info_list")[0].select(" .cast_txt")[0].get_text()
    
    if (mise[-2:] == "나쁨") or (choMise[-2:] == "나쁨") or (ozone[-2:] == "나쁨"):
        mask = "\n오늘은 마스크를 써주세요.\n"
    else:
        mask = ""

        
    if uv[-2:] == "나쁨":
        chucheon = "\n선크림을 발라주세요.\n"
    else:
        chucheon = ""
        
    if int(minMax.replace("˚","").split("/")[1]) - int(minMax.replace("˚","").split("/")[0]) > 10:
        ilgyo = "\n오늘은 일교차가 심하니 감기 조심하세요.\n"
    else:
        ilgyo = ""
        
    return minMax, """오늘의 {0} 날씨 알림입니다.
{10}                            
최저/최고 기온 : {1} 
체감 온도 : {2}
기상예보 : {3}
자외선 : {4}
{9}                                      
공기청정도 알림입니다.
미세먼지 : {5}
초미세먼지 : {6}
오존 : {7}
{8}                                          """.format(dongName, minMax, sensible, castText, uv, mise, choMise, ozone, mask, chucheon, ilgyo)
    
def nowTemp(dongName):
    head = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="
    keywords = dongName
    tail = "+날씨&oquery=오늘의+날씨&tqi=UCGJgwp0JXVss4ffedNssssstHo-344270"
    url = head + keywords + tail
    html = requests.get(url).text
    bs = BeautifulSoup(html, "lxml")
    now = bs.select_one(" .info_temperature").get_text().replace("도씨","")
    return now, "현재 기온은 {}입니다.\n\n".format(now[:-1])

def sendMail(me, you, msg, target, minMax):
    smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp.login(me, 'woigjcbmokllpyxm') ####
    msg = MIMEText(msg)
    msg['Subject'] = '오늘의 {0} 기온 : {1}'.format(target,minMax)###
    smtp.sendmail(me, you, msg.as_string())
    smtp.quit()

def sendMail_e6(me, you, msg, target, now):
    smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp.login(me, 'woigjcbmokllpyxm') ####
    msg = MIMEText(msg)
    msg['Subject'] = '현재의 {0} 기온 : {1}'.format(target,now)###
    smtp.sendmail(me, you, msg.as_string())
    smtp.quit()    

def quotes():
    proQuotes = "https://www.junauza.com/2010/12/top-50-programming-quotes-of-all-time.html"
    funQuotes = "https://www.junauza.com/2010/02/top-50-funny-computer-quotes.html"
    linQuotes = "https://www.junauza.com/2008/01/top-50-linux-quotes-of-all-time.html"
    p = re.compile('[0-9]+. [^0-9]+')

    html = requests.get(proQuotes).text
    bs = BeautifulSoup(html, "lxml")
    now = bs.select_one("#post-body-1882557240479875283")

    pro = now.text
    pro = pro[pro.find('50.'):-100]
    pro = pro.replace("\n","")
    pro = p.findall(pro)

    html = requests.get(funQuotes).text
    bs = BeautifulSoup(html, "lxml")
    now = bs.select_one("#post-body-6733972750347481628")

    fun = now.text
    fun = fun[fun.find('50. '):-225]
    fun = fun.replace("\n","")
    fun = p.findall(fun)

    html = requests.get(linQuotes).text
    bs = BeautifulSoup(html, "lxml")
    now = bs.select_one("#post-body-733398185143771750")

    lin = now.text
    lin = lin[lin.find('50. '):-222]
    lin = lin.replace("\n","")
    lin = p.findall(lin)

    listFunny = []
    listFunny.extend(pro)
    listFunny.extend(lin)
    listFunny.extend(fun)
    listFunny = listFunny[:-16] + listFunny[-8:]

    listFunny[57] = listFunny[57] + listFunny[58]
    listFunny.pop(58)

    listFunny[60] = listFunny[60] + "0"
    listFunny[61] = listFunny[61][1:]

    listFunny[65] = listFunny[65] + listFunny[66]
    listFunny.pop(66)
    print(len(listFunny))

    return listFunny
    
def mission():
    target = "가산동"
    nowGion, now = nowTemp(target)
    minMax, today = todayWeather(target)
    sendMail('yunjisag854@gmail.com', 'mroboedu@gmail.com',now + today, target, minMax)

def mission_e6():
    time = datetime.now().hour + 9
    if time > 24:
        time -= 24
    i = random.randrange(len(listFunny))
    ment = "{0} 하하하\n".format(listFunny[i][3:])
    if time == 7:
        ment = ment + "\n좋은 아침입니다. 행복한 하루 되세요."
    elif time == 13:
        ment = ment + "\n오늘 오전은 어떠셨나요? 벌써 절반이 지났습니다. 화이팅!"
    elif time == 19:
        ment = ment + "\n오늘 하루도 수고하셨습니다. 즐거운 저녁시간 되세요."
    else:
        ment = "{0} 하하하\n".format(listFunny[i][3:])
    target = "가산동"
    nowGion, now = nowTemp(target)
    msg = ment + "\n" + now
    sendMail_e6('yunjisag854@gmail.com', 'mroboedu@gmail.com', msg, target, nowGion)

In [ ]:
listFunny = quotes()
while True:
    time.sleep(3600)
    if datetime.today().hour == 7:
        while True:
            mission()
            for i in range(2):
                time.sleep(21600)
                mission_e6()
            time.sleep(21600)
            print("I`m working... {0}".format(datetime.today()))

149


# Testing

In [88]:
while True:
    mission()
    for i in range(3):
        time.sleep(6)
        mission_e6()
    time.sleep(6)
    print("I`m working... {0}".format(datetime.today()))

I`m working... 2020-02-21 01:51:41.385022
I`m working... 2020-02-21 01:52:20.108551
I`m working... 2020-02-21 01:52:58.837180
I`m working... 2020-02-21 01:53:37.071428
I`m working... 2020-02-21 01:54:15.624702
I`m working... 2020-02-21 01:54:55.332631


KeyboardInterrupt: 